# TO RUN : importS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
sns.set(font_scale = 1.3, rc = {'figure.figsize':(10,6)})
sns.set_palette('colorblind')

# SELECT DATA OF INTEREST 
As the project focus on the debate on the right to bear arms in the USA, the first task consist on selecting the data related to this topic. To do so, we used a lexical field related to the topic, and select only the quotations that contain one or more words of this lexical field. <br> The lexical field determined is the set of the following words : 'gun','firearm','mass shooting','2nd Amendment','murder','homicide','gun shot','armed robbery','rifles','Second Amendment','Columbine', 'gun control'. <br> <br>
The selected quotes are then stored in a new data file named `quotes-20__-extended.json.bz2` in form of a dataframe with some new columns. The columns added contain information about the speakers (gender, nationality, occupations, age (computed from the date of birth), ethnic group, party and religion). Such information are taken from the second dataset `speaker_attributes.parquet` itself built from wikidata information. Quotations that are not related to any seaker are not kept. <br> <br>
As such selection of data requires a lot of time, we decided to only treat the quotations of 2017 for this part of the project. The corresponding file having a weight of 5 Go, it contains way enough quotation to perform first statistic tests and check if our project is feasible. <br>
This is also why we decide to save the most information possible about the speaker. <br> <br>

In [ ]:
lexical_field = ['gun','firearm','mass shooting','2nd Amendment','homicide','gun shot','armed robbery','rifles','Second Amendment','Columbine', 'gun control']

speakers = pd.read_parquet(data_folder + 'speaker_attributes.parquet')
label = pd.read_csv(data_folder + 'wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

In [ ]:
path_to_file = data_folder + 'quotes-2017.json.bz2' 
path_to_out = data_folder + 'quotes-2017-extended-new.json.bz2'

iter = 0
nb_occ = 0

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample

            iter += 1
            if instance['numOccurrences'] is not None:
                nb_occ += instance['numOccurrences']

            if (iter % 100000 == 0):
                print('nombre de citations lus: {}'.format(iter))

            if any(substring.lower() in instance['quotation'].lower() for substring in lexical_field) and instance['qids'] != []: #We keep only quotation containing words of the lexical field and where there is a speaker
                speaker = speakers.loc[speakers['id'] == instance['qids'][0]].squeeze()

                #We add nationality
                if speaker.nationality is not None:
                    instance['nationality'] = []
                    for i in speaker['nationality']:
                        nat = label.loc[i]['Label']
                        instance['nationality'].append(nat)
                else:
                    instance['nationality'] = None
                    
                #We add the gender
                if speaker.gender is not None:
                    instance['gender'] = []
                    for i in speaker['gender']:
                        gend = label.loc[i]['Label']
                        instance['gender'].append(gend)
                else:
                    instance['gender'] = None

                #We add the occupations
                if speaker.occupation is not None:
                    instance['occupation'] = []
                    for i in speaker['occupation']:
                        occ = label.loc[i]['Label']
                        instance['occupation'].append(occ)
                else:
                    instance['occupation'] = None

                #We add the date of birth
                try:
                    born = datetime.strptime(speaker.date_of_birth[0][1:11], "%Y-%m-%d").date()
                    today = date.today()
                    age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
                except:
                    age = None
                instance['age'] = age

                #We add the ethnic group
                if speaker.ethnic_group is not None:
                    instance['ethnic_group'] = []
                    for i in speaker['ethnic_group']:
                        ethnic = label.loc[i]['Label']
                        instance['ethnic_group'].append(ethnic)
                else:
                    instance['ethnic_group'] = None

                #We add the party
                if speaker.party is not None:
                    instance['party'] = []
                    for i in speaker['party']:
                        part = label.loc[i]['Label']
                        instance['party'].append(part)
                else:
                    instance['party'] = None

                #We add the religion
                if speaker.religion is not None:
                    instance['religion'] = []
                    for i in speaker['religion']:
                        relig = label.loc[i]['Label']
                        instance['religion'].append(relig)
                else:
                    instance['religion'] = None

                d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

                # to test
                # if iter > 10000:
                #     break

print('iter = {i}'.format(i = iter))
print('nb_occ = {n}'.format(n = nb_occ))

**Useful statistics :**   
26 611 588 "different" quotes were extracted from 2017 newpapers.  
Some of these quotes being mentionned in several articles, the total number of quotes found in the newspapers is 136 326 717.